In [5]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point

Same pool of trials for all participants (static), with different shuffling (dynamic).

---

# Static Layout Pool
This refers to the generation of the layout pool, which will is the same for ALL participants.

## Generation

- Given N trials, we produce  N/2 pairs of layouts.
    - Between these pairs, we keep the same coordinates for the points, and the same order of navigation.
    - One of these pairs is the no-boundary-show, the other is the boundary-shown. The layout is rotated between these two.
    - Per layout, we have 5 objects and an ending location. The first object is also the starting location and might be asked.
    - We do not want any 2 objects to be too close to each other.
    - We do not want any path to be too short. We do not have an upper bound on the travel distance.
    - We want to to make sure that there is at least one object on both sides.

In [20]:
N_POINTS_LAYOUT = 6 # 5 objects, one of which is the starting location, and an ending location
N_LAYOUTS = 50 # generate this many layouts. each layout produces a pair (visible/invisible boundary)

# these are currently arbitrary units that we can scale up and down as needed
# w=5, l=10 represents a 1 by 2 rectangle room
WIDTH = 5 
LENGTH = 10

# this is the minimum distance any two objects (not a path) can have. 
# used to prevent objects appearing too close and confusing participants
MIN_DIST_OBJECTS = 1.5

# this is the minimum path distance. no subsequent object should appear closer than this.
# greater than MIN_DIST_OBJECTS because objects appearing too close also means they are visited too soon and not enough time to learn that object
MIN_DIST_PATH = 2.3

In [74]:
def generate_random_layout():
    return [Point.random_rect(WIDTH, LENGTH) for p in range(N_POINTS_LAYOUT)]

def has_valid_margin(p):
    buffer = 0.05
    x_lo = buffer * WIDTH
    x_hi = (1 - buffer) * WIDTH
    
    y_lo = buffer * LENGTH
    y_hi = (1 - buffer) * LENGTH
    
    border_lo = (LENGTH / 2) - LENGTH * buffer
    border_hi = (LENGTH / 2) + LENGTH * buffer
    
    bufferFromBoundary = (x_lo < p.x < x_hi) and (y_lo < p.y < y_hi)
    bufferFromBorder = not (border_lo < p.y < border_hi)
    
    return bufferFromBoundary and bufferFromBorder

def is_valid_layout(lay):
    
    # check all distance constraints, (dist between objects and path distances)
    for i in range(len(lay)):
        for j in range(i):
            # if these points are adjacent, check for path distance condition, otherwise object distance condition
            lower_bound = MIN_DIST_PATH if j == i - 1 else MIN_DIST_OBJECTS
            if lay[i].dist(lay[j]) <= lower_bound:
                return False
    
    # check that there is at least one object on each side
    # note that this checks for objects ONLY (that is; not the last point which is the ending/task location)
    objects_on_lower_side = sum([1 if l.y < LENGTH / 2 else 0 for l in lay[:N_POINTS_LAYOUT-1]])
    if not ( 1 <= objects_on_lower_side < N_POINTS_LAYOUT-1):
        # either 0 or N_POINTS_LAYOUT-1 on this side, which means the reverse for the other side. not valid!
        return False
    
    for p in lay:
        if not has_valid_margin(p):
            return False
    
    return True

def generate_valid_layout():
    candidate = generate_random_layout()
    c = 0
    while not is_valid_layout(candidate):
        c += 1
        if c == 1e5:
            print("can not find valid layout!")
            return None
        candidate = generate_random_layout()
    return candidate

In [75]:
layouts = []
for l in range(N_LAYOUTS):
    layouts.append(generate_valid_layout())

In [76]:
print(layouts)

[[P(2.03, 6.42), P(0.53, 8.59), P(4.61, 6.44), P(4.05, 8.98), P(1.43, 3.73), P(3.84, 3.47)], [P(4.14, 3.26), P(3.32, 9.27), P(0.39, 7.9), P(3.84, 7.45), P(0.81, 2.38), P(2.52, 6.01)], [P(3.24, 7.48), P(4.36, 1.17), P(1.14, 2.3), P(0.25, 6.2), P(2.95, 2.79), P(3.85, 8.87)], [P(4.67, 3.17), P(4.57, 7.61), P(1.22, 9.03), P(4.55, 5.7), P(4.2, 9.32), P(0.92, 2.67)], [P(1.6, 9.02), P(2.36, 2.57), P(1.41, 6.02), P(1.59, 0.63), P(3.12, 4.0), P(3.94, 9.21)], [P(1.01, 3.36), P(2.11, 7.79), P(4.33, 3.87), P(1.18, 1.18), P(3.65, 0.95), P(3.69, 7.81)], [P(2.96, 6.17), P(1.07, 0.89), P(3.62, 9.0), P(4.68, 6.56), P(2.15, 4.08), P(0.32, 2.42)], [P(2.28, 9.41), P(4.63, 1.09), P(4.05, 8.86), P(2.83, 0.64), P(0.73, 7.67), P(0.26, 1.95)], [P(2.7, 6.7), P(4.58, 0.54), P(3.31, 4.31), P(1.97, 8.2), P(4.7, 6.33), P(2.43, 1.22)], [P(2.08, 1.92), P(3.96, 6.1), P(3.64, 2.9), P(0.72, 5.5), P(4.5, 0.57), P(1.02, 0.81)], [P(3.43, 2.27), P(0.25, 6.08), P(0.6, 8.73), P(2.33, 7.04), P(1.93, 3.65), P(4.06, 7.05)], [P(2

## Checks
- We want to check for the following distributions:
    - Pointing ground-truths -> compare those in the same side and across the boundary.
    - How much they have traveled
    - Distances between any two objects
    
- And the following are nice sanity checks to have:
    - Plot of all layouts
    - Plot of all points

In [ ]:
# Ground Truth for Pointing Tasks (GTPT)
gtpt_same = []
gtpt_cross = []

for lay in layouts:
    last = lay[-1]
    last_side = last.y < LENGTH / 2
    
    for obj in lay[:-1]:
        is_on_same_side = last_side == (obj.y < LENGTH / 2)
        dist = last.dist(obj)
        if is_on_same_side:
            gtpt_same.append(dist)
        else:
            gtpt_cross.append(dist)

gtpt_all = [gtpt_same, gtpt_cross]
print(len(gtpt_same), "same side; ", len(gtpt_cross), "across boundary")

In [ ]:
plt.hist(gtpt_all, bins = 24, range = [0, max(gtpt_same + gtpt_cross)] ,histtype='step', stacked=True, fill=False, label = ["Same Side", "Across Boundary"])
plt.legend(prop={'size': 10})
plt.title("Distribution of Pointing Task Ground-Truth Distances by Object Side")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize = (9, 12))
for i in range(N_POINTS_LAYOUT):
    ith_ps = [layouts[j][i] for j in range(N_LAYOUTS)]
    xs, ys = [p.x for p in ith_ps], [p.y for p in ith_ps]
    axs[i//3, i%3].scatter(xs, ys)
    axs[i//3, i%3].set_title(f"Distribution of {i}th points")

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.

## Checks

- At least K = 5 trials between trials of the same pairing.
- We make sure that approx. half the pairs happen in A->B, and the other half happens in B->A
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
- We want to check the number of objects within the same cells across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.